In [19]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.metrics import auc, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import src.graphics as graphics

In [20]:
VERSION = 2
MODEL_BASE_NAME = "MegaClassifier_a_EfficientNetB5"

DATASET_CSV = os.path.abspath(
    "./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv"
)
DATASET_PATH = os.path.dirname(DATASET_CSV)

In [21]:
dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset["file_name"] = dataset["file_name"].apply(
    lambda x: os.path.join(DATASET_PATH, x)
)
dataset["binary_label"] = dataset["binary_label"].astype(str)

train_dataset = dataset[dataset["subset"] == "train"]
validationtrain_dataset = dataset[dataset["subset"] == "validation"]
test_dataset = dataset[dataset["subset"] == "test"]

EPOCHS = 10
IMAGE_SIZE = (456, 456)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [22]:
OPTIMIZERS = {"Adam": tf.keras.optimizers.legacy.Adam()}

In [23]:
num_class_0 = (train_dataset["binary_label"] == 0).sum()
num_class_1 = (train_dataset["binary_label"] == 1).sum()

total_samples = num_class_0 + num_class_1

weight_for_0 = total_samples / (2 * num_class_0)
weight_for_1 = total_samples / (2 * num_class_1)

class_weights = {0: weight_for_0, 1: weight_for_1}

/var/folders/fy/xnmbh7m55bjghqdjfps53_j00000gn/T/ipykernel_74470/1556420298.py:6: RuntimeWarning: invalid value encountered in scalar divide
  weight_for_0 = total_samples / (2 * num_class_0)
/var/folders/fy/xnmbh7m55bjghqdjfps53_j00000gn/T/ipykernel_74470/1556420298.py:7: RuntimeWarning: invalid value encountered in scalar divide
  weight_for_1 = total_samples / (2 * num_class_1)


In [24]:
LOSS_FUNCTIONS = {
    "BinaryCrossentropy": tf.keras.losses.BinaryCrossentropy(),
    "BinaryCrossentropyWeighted": tf.keras.losses.BinaryCrossentropy(),
}

In [25]:
METRICS = [
    BinaryAccuracy(name="accuracy"),
    Precision(name="precision"),
    Recall(name="recall"),
    AUC(name="auc"),
]

In [26]:
BATCH_SIZE = 16

In [ ]:
SUBVERSION = 0
for LOSS_FUNCTION in LOSS_FUNCTIONS:
    LOGS_PATH = os.path.abspath(
        f"./logs/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    MODELS_PATH = os.path.abspath(
        f"./models/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    REPORTS_PATH = os.path.abspath(
        f"./reports/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )

    MODEL_COMPLETE_NAME = f"{MODEL_BASE_NAME} v{VERSION}.{SUBVERSION}"

    train_datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    )
    train_images = train_datagen.flow_from_dataframe(
        dataframe=train_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )

    datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    )
    validation_images = datagen.flow_from_dataframe(
        dataframe=validationtrain_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    test_images = datagen.flow_from_dataframe(
        dataframe=test_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=False,
        seed=SEED,
    )

    pretrained_model = tf.keras.applications.EfficientNetB5(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    pretrained_model.trainable = False

    model = tf.keras.Sequential(
        [
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(1, activation="sigmoid"),
        ],
        name=f"{MODEL_BASE_NAME}_v{VERSION}.{SUBVERSION}",
    )

    model.compile(
        optimizer=OPTIMIZERS["Adam"],
        loss=LOSS_FUNCTIONS[LOSS_FUNCTION],
        metrics=METRICS,
    )

    history = model.fit(
        train_images,
        epochs=EPOCHS,
        validation_data=validation_images,
        callbacks=[TensorBoard(log_dir=LOGS_PATH)],
        class_weight=class_weights
        if LOSS_FUNCTION == "BinaryCrossentropyWeighted"
        else None,
    )

    dataframe = pd.DataFrame(history.history)
    history_path = os.path.join(LOGS_PATH, f"history_v{VERSION}.{SUBVERSION}.csv")
    dataframe.to_csv(history_path, sep=";", index=False)

    os.makedirs(REPORTS_PATH, exist_ok=True)
    accuracy_chart = graphics.create_training_accuracy_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    accuracy_chart.write_image(f"{REPORTS_PATH}/accuracy_v{VERSION}.{SUBVERSION}.png")

    loss_chart = graphics.create_training_loss_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    loss_chart.write_image(f"{REPORTS_PATH}/loss_v{VERSION}.{SUBVERSION}.png")

    os.makedirs(MODELS_PATH, exist_ok=True)
    model.save(MODELS_PATH)

    results = model.evaluate(test_images)
    metric_names = history.model.metrics_names
    evaluation_results = {
        ("test_" + name): value for name, value in zip(metric_names, results)
    }

    evaluation = pd.DataFrame([evaluation_results])
    evaluation.to_csv(
        os.path.join(LOGS_PATH, f"evaluation_v{VERSION}.{SUBVERSION}.csv"),
        sep=";",
        index=False,
    )

    y_pred_prob = model.predict(test_images)
    y_true = test_images.labels

    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    roc_curve_chart = graphics.create_roc_curve_chart(
        fpr=fpr,
        tpr=tpr,
        roc_auc=roc_auc,
        model_name=MODEL_COMPLETE_NAME,
    )
    roc_curve_chart.write_image(f"{REPORTS_PATH}/roc_curve_v{VERSION}.{SUBVERSION}.png")

    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    # y_pred_prob = y_pred_prob.flatten()
    y_pred_class = (y_pred_prob > optimal_threshold).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    # Crear el gráfico correctamente
    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_optimal_v{VERSION}.{SUBVERSION}.png"
    )

    y_pred_class = (y_pred_prob > 0.5).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_v{VERSION}.{SUBVERSION}.png"
    )

    SUBVERSION += 1
    print("\n\n")


In [28]:
LOSS_FUNCTIONS = {
    "BinaryFocalCrossentropy": lambda x, y: tf.keras.losses.BinaryFocalCrossentropy(
        alpha=x, gamma=y
    ),
    "SigmoidFocalCrossEntropy": lambda x, y: tfa.losses.SigmoidFocalCrossEntropy(
        alpha=x, gamma=y
    ),
}

ALPHAS = [0.25, 0.4, 0.6, 0.75]
GAMMAS = [1.0, 2.0, 3.0]

In [29]:
SUBVERSION = 2
ALPHA_SUBVERSION = 0
GAMMA_SUBVERSION = 0

for LOSS_FUNCTION in LOSS_FUNCTIONS:
    for ALPHA in ALPHAS:
        for GAMMA in GAMMAS:
            LOGS_PATH = os.path.abspath(
                f"./logs/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}"
            )
            MODELS_PATH = os.path.abspath(
                f"./models/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}"
            )
            REPORTS_PATH = os.path.abspath(
                f"./reports/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}"
            )

            MODEL_COMPLETE_NAME = f"{MODEL_BASE_NAME} v{VERSION}.{SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}"

            train_datagen = ImageDataGenerator(
                preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
            )
            train_images = train_datagen.flow_from_dataframe(
                dataframe=train_dataset,
                x_col="file_name",
                y_col="binary_label",
                target_size=IMAGE_SIZE,
                batch_size=BATCH_SIZE,
                class_mode="binary",
                shuffle=True,
                seed=SEED,
            )

            datagen = ImageDataGenerator(
                preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
            )
            validation_images = datagen.flow_from_dataframe(
                dataframe=validationtrain_dataset,
                x_col="file_name",
                y_col="binary_label",
                target_size=IMAGE_SIZE,
                batch_size=BATCH_SIZE,
                class_mode="binary",
                shuffle=True,
                seed=SEED,
            )
            test_images = datagen.flow_from_dataframe(
                dataframe=test_dataset,
                x_col="file_name",
                y_col="binary_label",
                target_size=IMAGE_SIZE,
                batch_size=BATCH_SIZE,
                class_mode="binary",
                shuffle=False,
                seed=SEED,
            )

            pretrained_model = tf.keras.applications.EfficientNetB5(
                weights="imagenet",
                include_top=False,
                input_shape=IMAGE_SHAPE,
            )
            pretrained_model.trainable = False

            model = tf.keras.Sequential(
                [
                    pretrained_model,
                    tf.keras.layers.GlobalAveragePooling2D(),
                    tf.keras.layers.Dense(1, activation="sigmoid"),
                ],
                name=f"{MODEL_BASE_NAME}_v{VERSION}.{SUBVERSION}",
            )

            model.compile(
                optimizer=OPTIMIZERS["Adam"],
                loss=LOSS_FUNCTIONS[LOSS_FUNCTION](x=ALPHA, y=GAMMA),
                metrics=METRICS,
            )

            history = model.fit(
                train_images,
                epochs=EPOCHS,
                validation_data=validation_images,
                callbacks=[TensorBoard(log_dir=LOGS_PATH)],
            )

            dataframe = pd.DataFrame(history.history)
            history_path = os.path.join(
                LOGS_PATH,
                f"history_v{VERSION}.{SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}.csv",
            )
            dataframe.to_csv(history_path, sep=";", index=False)

            os.makedirs(REPORTS_PATH, exist_ok=True)
            accuracy_chart = graphics.create_training_accuracy_chart(
                history_path=history_path,
                model_name=MODEL_COMPLETE_NAME,
            )
            accuracy_chart.write_image(
                f"{REPORTS_PATH}/accuracy_v{VERSION}.{SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}.png"
            )

            loss_chart = graphics.create_training_loss_chart(
                history_path=history_path,
                model_name=MODEL_COMPLETE_NAME,
            )
            loss_chart.write_image(
                f"{REPORTS_PATH}/loss_v{VERSION}.{SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}.png"
            )

            os.makedirs(MODELS_PATH, exist_ok=True)
            model.save(MODELS_PATH)

            results = model.evaluate(test_images)
            metric_names = history.model.metrics_names
            evaluation_results = {
                ("test_" + name): value for name, value in zip(metric_names, results)
            }

            evaluation = pd.DataFrame([evaluation_results])
            evaluation.to_csv(
                os.path.join(
                    LOGS_PATH,
                    f"evaluation_v{VERSION}.{SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}.csv",
                ),
                sep=";",
                index=False,
            )

            y_pred_prob = model.predict(test_images)
            y_true = test_images.labels

            fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
            roc_auc = auc(fpr, tpr)

            roc_curve_chart = graphics.create_roc_curve_chart(
                fpr=fpr,
                tpr=tpr,
                roc_auc=roc_auc,
                model_name=MODEL_COMPLETE_NAME,
            )
            roc_curve_chart.write_image(
                f"{REPORTS_PATH}/roc_curve_v{VERSION}.{SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}.png"
            )

            optimal_idx = np.argmax(tpr - fpr)
            optimal_threshold = thresholds[optimal_idx]

            # y_pred_prob = y_pred_prob.flatten()
            y_pred_class = (y_pred_prob > optimal_threshold).astype(int)
            conf_matrix = confusion_matrix(y_true, y_pred_class)
            conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

            # Crear el gráfico correctamente
            confusion_matrix_chart = graphics.create_confusion_matrix_chart(
                conf_matrix=conf_matrix,
                conf_matrix_text=conf_matrix_text,
                model_name=MODEL_COMPLETE_NAME,
            )
            confusion_matrix_chart.write_image(
                f"{REPORTS_PATH}/confusion_matrix_optimal_v{VERSION}.{SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}.png"
            )

            y_pred_class = (y_pred_prob > 0.5).astype(int)
            conf_matrix = confusion_matrix(y_true, y_pred_class)
            conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

            confusion_matrix_chart = graphics.create_confusion_matrix_chart(
                conf_matrix=conf_matrix,
                conf_matrix_text=conf_matrix_text,
                model_name=MODEL_COMPLETE_NAME,
            )
            confusion_matrix_chart.write_image(
                f"{REPORTS_PATH}/confusion_matrix_v{VERSION}.{SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}.png"
            )

            GAMMA_SUBVERSION += 1

        GAMMA_SUBVERSION = 0
        ALPHA_SUBVERSION += 1

    GAMMA_SUBVERSION = 0
    ALPHA_SUBVERSION = 0

    SUBVERSION += 1
    print("\n\n")


Found 17906 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Epoch 1/10


2025-03-04 06:18:53.337282: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1120/1120 [==============================] - ETA: 0s - loss: 0.0986 - accuracy: 0.8051 - precision: 0.9430 - recall: 0.7777 - auc: 0.7764

2025-03-04 06:36:56.376400: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1120/1120 [==============================] - 1338s 1s/step - loss: 0.0986 - accuracy: 0.8051 - precision: 0.9430 - recall: 0.7777 - auc: 0.7764 - val_loss: 0.0754 - val_accuracy: 0.9440 - val_precision: 0.9558 - val_recall: 0.9598 - val_auc: 0.9854
Epoch 2/10
1120/1120 [==============================] - 1328s 1s/step - loss: 0.0822 - accuracy: 0.9319 - precision: 0.9527 - recall: 0.9553 - auc: 0.9797 - val_loss: 0.0723 - val_accuracy: 0.9459 - val_precision: 0.9735 - val_recall: 0.9439 - val_auc: 0.9860
Epoch 3/10
1120/1120 [==============================] - 1328s 1s/step - loss: 0.0762 - accuracy: 0.9364 - precision: 0.9577 - recall: 0.9561 - auc: 0.9826 - val_loss: 0.0706 - val_accuracy: 0.9496 - val_precision: 0.9726 - val_recall: 0.9506 - val_auc: 0.9863
Epoch 4/10
1120/1120 [==============================] - 1328s 1s/step - loss: 0.0732 - accuracy: 0.9397 - precision: 0.9589 - recall: 0.9596 - auc: 0.9839 - val_loss: 0.0689 - val_accuracy: 0.9510 - val_precision: 0.9617 - val_reca

...

1120/1120 [==============================] - 1464s 1s/step - loss: 0.0123 - accuracy: 0.9092 - precision: 0.9083 - recall: 0.9728 - auc: 0.9718 - val_loss: 0.0093 - val_accuracy: 0.9442 - val_precision: 0.9532 - val_recall: 0.9630 - val_auc: 0.9857
Epoch 2/10
1120/1120 [==============================] - 1424s 1s/step - loss: 0.0094 - accuracy: 0.9197 - precision: 0.9164 - recall: 0.9807 - auc: 0.9790 - val_loss: 0.0082 - val_accuracy: 0.9377 - val_precision: 0.9321 - val_recall: 0.9771 - val_auc: 0.9871
Epoch 3/10
1120/1120 [==============================] - 1425s 1s/step - loss: 0.0087 - accuracy: 0.9227 - precision: 0.9194 - recall: 0.9813 - auc: 0.9819 - val_loss: 0.0085 - val_accuracy: 0.9351 - val_precision: 0.9283 - val_recall: 0.9774 - val_auc: 0.9866
Epoch 4/10
1120/1120 [==============================] - 1424s 1s/step - loss: 0.0082 - accuracy: 0.9294 - precision: 0.9271 - recall: 0.9816 - auc: 0.9839 - val_loss: 0.0087 - val_accuracy: 0.9440 - val_precision: 0.9491 - val_reca

INFO:tensorflow:Assets written to: /Users/litto/WORKSPACE/MegaClassifier/models/MegaClassifier_a_EfficientNetB5/v2/v2.3.3.2/assets


268/268 [==============================] - 272s 1s/step - loss: 0.0069 - accuracy: 0.9522 - precision: 0.9550 - recall: 0.9736 - auc: 0.9913


2025-03-08 08:34:36.853464: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


268/268 [==============================] - 284s 1s/step



